In [0]:
# Set to false if you are not running
# this notebook in Google Colaboratory
run_on_colab = True

In [2]:
if(run_on_colab):
  from google.colab import drive
  # This will prompt for authorization.
  drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
!pip install music21;

### Upload folk_C_midi.zip to files, using the arrow on the left
 


In [5]:
!unzip midi_files_5000.zip

Archive:  midi_files_5000.zip
  inflating: sessiontune777.mid      
   creating: __MACOSX/
  inflating: __MACOSX/._sessiontune777.mid  
  inflating: sessiontune3403.mid     
  inflating: __MACOSX/._sessiontune3403.mid  
  inflating: sessiontune1214.mid     
  inflating: __MACOSX/._sessiontune1214.mid  
  inflating: sessiontune1572.mid     
  inflating: __MACOSX/._sessiontune1572.mid  
  inflating: sessiontune3365.mid     
  inflating: __MACOSX/._sessiontune3365.mid  
  inflating: sessiontune3371.mid     
  inflating: __MACOSX/._sessiontune3371.mid  
  inflating: sessiontune1566.mid     
  inflating: __MACOSX/._sessiontune1566.mid  
  inflating: sessiontune1200.mid     
  inflating: __MACOSX/._sessiontune1200.mid  
  inflating: sessiontune2709.mid     
  inflating: __MACOSX/._sessiontune2709.mid  
  inflating: sessiontune5066.mid     
  inflating: __MACOSX/._sessiontune5066.mid  
  inflating: sessiontune4378.mid     
  inflating: __MACOSX/._sessiontune4378.mid  
  inflating: sessiontune

In [0]:
# Importing dependencies
import glob
import pickle
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from music21 import converter, instrument, note, chord, stream, pitch
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU, LSTM, Activation, CuDNNLSTM, CuDNNGRU, BatchNormalization
from tensorflow.python.keras import utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import metrics, optimizers
from tensorflow.keras.regularizers import L1L2

from tensorflow.keras import backend as K
from tensorflow.python.keras.layers import Layer
from tensorflow.keras import initializers, regularizers, constraints

In [0]:
file = "folk_C_midis/sessiontune10.mid"
midi = converter.parse(file)
notes_to_parse = midi.flat.notes
for element in notes_to_parse[:10]:
  print(element, element.offset)

<music21.note.Note E> 0.0
<music21.note.Note A> 0.5
<music21.note.Note A> 1.0
<music21.note.Note G> 1.5
<music21.note.Note A> 2.0
<music21.note.Note C> 2.5
<music21.note.Note E> 3.0
<music21.note.Note F> 3.5
<music21.note.Note A> 4.5
<music21.note.Note C> 5.0


In [8]:
# notess = list of list of notes/chords
# notes = list of notes/chords in each song
# this is to prevent input sequences that span across songs

lowest_note_midi_number = 60 # Transpose all notes to be above/equal to C4
n_vocab = 24 # only using notes from [C4-C6)
notess = []
for i,file in enumerate(glob.glob("*.mid")):
  notes = []
  print('\r', 'Parsing file ', i, " ",file, end='')
  midi = converter.parse(file)
  notes_to_parse = None
  try: # file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse() 
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes
    
  for element in notes_to_parse:
    # ps = get all pitches from note/chord
    ps = []
    if isinstance(element, note.Note):
      ps.append(element.pitch)
    elif isinstance(element, chord.Chord):
      for p in element.pitches:
        ps.append(p)
        
    # n = single note/chord, initialize to all zeros
    # iterate over pitches extracted, set cells to 1 in n
    n = [0] * 24 # [C4-C6)    
    for p in ps:
      if p.octave < 4:
        p.octave = 4
      elif p.octave > 5:
        p.octave = 5
      n[p.midi-lowest_note_midi_number] = 1 # 60 = C4's midi number
    notes.append(n)
    
  notess.append(notes)
with open('folk_notess_one_hot', 'wb') as filepath:
  pickle.dump(notess, filepath)

 Parsing file  5064   sessiontune372.mid

In [0]:
with open('rock_notess', 'rb') as filepath:
  notess = pickle.load(filepath)

In [9]:
# Check notes
print(notess[0])

[[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

We obtain the number of different notes in our dataset, because this will be the **number of possible output classes**  of our model.

In [0]:
# Count different possible outputs
# flatten = [item for sublist in notess for item in sublist]
# n_vocab = (len(set(flatten)))
n_vocab = 24

In [11]:
sequence_length = 80
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for notes in notess:
  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length] # Size sequence_length, 24
    sequence_out = notes[i + sequence_length]  # Size 24
    network_input.append(sequence_in)
    network_output.append(sequence_out)
n_patterns = len(network_input)

# Cuz lstm layers only take in np arrays
network_input = np.array(network_input)
network_output = np.array(network_output)
print(network_input[0][:3])

[[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


Let's see the new network_input size

In [12]:
network_input.shape

(552364, 80, 24)

In [0]:
# https://www.kaggle.com/qqgeogor/keras-lstm-attention-glove840b-lb-0-043
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1].value,),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight(shape=(input_shape[1].value,),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

### Multilabel classification problem
top k categorical accuracy: single accuracy doesnt make sense

binary cross entropy: each cell is treated is its own independent cell, binary classification for each cell

sigmoid activation for 0-1 output


In [0]:
def top_3_categorical_accuracy(y_true, y_pred):
  return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

In [0]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    reg = L1L2(0, 0)
    model.add(GRU(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        return_sequences=True,
        dropout=0.3, recurrent_dropout=0.3
    ))
    model.add(CuDNNGRU(512, return_sequences=True, kernel_regularizer=reg))
    model.add(CuDNNGRU(512, return_sequences=True, kernel_regularizer=reg))
    model.add(Attention(80))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[top_3_categorical_accuracy])
    return model

In [18]:
model = create_network(network_input,n_vocab)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 80, 512)           824832    
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (None, 80, 512)           1575936   
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 80, 512)           1575936   
_________________________________________________________________
attention (Attention)        (None, 512)               592       
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)    

In [0]:
# In case we want to use previously trained weights
# weights = ""
weights = "/content/drive/My Drive/save_points/"
if(len(weights)>0): 
  model.load_weights(weights)
  print('weights loaded')

weights loaded


In [19]:
filepath = "/content/drive/My Drive/save_points/{epoch:02d}-{loss:.4f}.h5"

checkpoint = ModelCheckpoint(filepath, monitor='loss',verbose=0,
                             save_best_only=True,mode='min')

callbacks_list = [checkpoint]
history = model.fit(network_input, np.array(network_output), initial_epoch=0, epochs=150, batch_size=256, 
          callbacks=callbacks_list, validation_split=0.2)

Train on 441891 samples, validate on 110473 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/150
 14848/441891 [>.............................] - ETA: 14:06 - loss: 0.1909 - top_3_categorical_accuracy: 0.3160

KeyboardInterrupt: ignored

In [0]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

NameError: ignored

### Running next two cells only if starting from scratch

In [0]:
# In case we want to use other previously trained weights
weights = "drive/My Drive/save_points/10-0.0788.h5"
if(len(weights)>0): model.load_weights(weights)

In [0]:
# Generate network input again
test_input = []
test_output = []
for notes in notess:
  for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    test_input.append([note_to_int[char] for char in sequence_in])
    test_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)

In [61]:
""" Generate notes from the neural network based on a sequence of notes """
# pick a random sequence from the input as a starting point for the prediction
play_note_threshold = 0.25

# 1. Use actual patterns as seed
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start][:sequence_length]
# 2. Use random patterns as seed
# pattern = np.random.binomial(1, 0.2, size=sequence_length*n_vocab).reshape((sequence_length, n_vocab))
# 3. Use user input pattern as seed 
# I-IV-V-I
# user_input = [['D4', 'F#4', 'A4', 'D5'], ['D4', 'F#4', 'A4', 'D5'], ['D4', 'F#4', 'A4', 'D5'], ['D4', 'F#4', 'A4', 'D5'], ['G4', 'B4', 'D5', 'G5'], ['G4', 'B4', 'D5', 'G5'], ['G4', 'B4', 'D5', 'G5'], ['G4', 'B4', 'D5', 'G5'], ['A4', 'C#5', 'E5', 'A5'], ['A4', 'C#5', 'E5', 'A5'], ['A4', 'C#5', 'E5', 'A5'], ['A4', 'C#5', 'E5', 'A5'], ['D4', 'F#4', 'A4', 'D5'], ['D4', 'F#4', 'A4', 'D5'], ['D4', 'F#4', 'A4', 'D5'], ['D4', 'F#4', 'A4', 'D5']]
# j = 0
# pattern = []
# for i in range(sequence_length):
# #   ps = get all pitches from note/chord
#   ps = []
#   input = user_input[j]
#   if len(input) > 1:
#     tempChord = chord.Chord(input)
#     for p in tempChord.pitches:
#       ps.append(p)   
#   elif len(input) == 1:
#     tempNote = note.Note(input[0])
#     ps.append(tempNote.pitch)
#   # n = single note/chord, initialize to all zeros
#   # iterate over pitches extracted, set cells to 1 in n
#   n = [0] * 24 # [C4-C6)    
#   for p in ps:
#     if p.octave < 4:
#       p.octave = 4
#     elif p.octave > 5:
#       p.octave = 5
#     n[p.midi-lowest_note_midi_number] = 1 # 60 = C4's midi number
#   pattern.append(n)
#   j += 1
#   if j % len(user_input) == 0:
#     j = 0

prediction_output = []
# generate 100 notes
for i,note_index in enumerate(range(100)):
  # Hacky code with lots of reshapes
  prediction_input = np.reshape(pattern, (1, len(pattern), n_vocab))
  prediction = model.predict(prediction_input, verbose=0)
  prediction = prediction.reshape((n_vocab,))
  transposed_midi_nums = np.argwhere(prediction > play_note_threshold)
  result = []
  for transposed_midi_num in transposed_midi_nums:
    result.append(pitch.Pitch(transposed_midi_num[0]+60).nameWithOctave)
  print('\r', 'Predicted ', i, " ",result, end='')
  prediction_output.append(result)
  
  prediction[np.argwhere(prediction > play_note_threshold)] = 1
  prediction[np.argwhere(prediction <= play_note_threshold)] = 0
  pattern = np.concatenate((pattern, prediction.reshape(1, prediction.shape[0])))
  pattern = pattern[1:len(pattern)]

 Predicted  99   ['E4', 'A4']

In [62]:
for pattern in prediction_output:
  print(pattern)

[]
['E4']
['G4']
['E4', 'A4']
[]
[]
['C5']
['A4', 'D5']
['E5']
['C5', 'D5']
['A4', 'C5']
['A4']
['G4']
['E4']
['C4', 'C5']
['E4']
['C4']
['D4', 'A4']
['E4']
['C4']
['D4', 'A4']
['C4', 'E4']
['G4', 'A4']
['A4']
['G4']
['E4']
['D4']
['C4', 'E4']
['E4']
['G4']
['E4', 'A4']
['E4', 'G4']
['A4']
['G4']
['E4', 'A4']
['G4']
['A4']
['G4']
['E4']
['G4']
['A4']
['G4']
['E4']
['D4', 'G4']
['C4']
['D4']
['C4', 'E4']
['G4']
['A4']
['G4', 'D5']
['D5']
[]
['C5']
['A4']
['G4']
['E4', 'A4']
['A4']
['G4']
['E4']
['C4', 'D4']
['D4']
['E4']
['C4', 'G4']
['D4']
['E4']
['C4', 'G4']
['D4', 'G4']
['E4', 'G4']
['G4']
['A4']
['G4', 'D5']
['D5']
['E5']
['C5']
['A4']
['G4']
['E4', 'A4']
['C4']
['D4']
['E4']
['C4']
['D4', 'A4']
['E4']
['G4']
['A4']
['G4']
['E4']
['D4', 'G4']
['D4', 'G4']
[]
['G4', 'A4']
['A4']
['G4']
['E4']
['D4']
['C4', 'E4']
['C4', 'E4']
[]
['G4']
['E4', 'A4']


In [63]:
offset = 0
output_notes = []
chord_names = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    if len(pattern) > 1:
      chord_notes = []
      for current_note in pattern:
        new_note = note.Note(current_note)
        new_note.storedInstrument = instrument.Piano()
        chord_notes.append(new_note)
      new_chord = chord.Chord(chord_notes)
#       new_chord.duration.quarterLength = duration
      new_chord.offset = offset
      output_notes.append(new_chord)
      chord_names.append(new_chord.commonName)
    # pattern is a note
    elif len(pattern) == 1:
        new_note = note.Note(pattern[0])
#         new_note.duration.quarterLength = duration
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    
    # increase offset each iteration so that notes do not stack
    offset += 0.5

midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'

### Look at chord progressions

In [64]:
print([c_name for i, c_name in enumerate(chord_names) if i == 0 or c_name != chord_names[i-1]])

['Perfect Fourth', 'Major Second', 'Minor Third', 'unison', 'Perfect Fifth', 'Major Third', 'Major Second', 'Major Third', 'Perfect Fourth', 'Minor Third', 'Perfect Fourth', 'Major Third', 'Perfect Fifth', 'Perfect Fourth', 'Major Second', 'Perfect Fifth', 'Perfect Fourth', 'Minor Third', 'Perfect Fifth', 'Perfect Fourth', 'Perfect Fifth', 'Perfect Fourth', 'Major Second', 'Major Third', 'Perfect Fourth']
